In [1]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
import re
import json


In [2]:
# loop through all files in the import directory
for filename in os.listdir('import'):
    # open the file
    with open('import/' + filename) as file:
        # array to store the content of the file line by line
        content = []
        # loop through each line in the file
        for line in file:
            # remove the newline character from the end of the line
            line = line.rstrip()
            # remove the ' \u200b' character from the end of the line
            line = line.replace(' \u200b', '')
            # add the line to the content array
            content.append(line)
                    

In [3]:
load_dotenv()
googleKey = os.getenv('googleKey')

# create a youtube object
youtube = build('youtube', 'v3', developerKey=googleKey)

In [4]:
def remove_time_tag(url):
    # Remove time tag if present in the URL
    return re.sub(r'[&?]t=\d+[hms]?(&?)', r'\1', url)

def get_video_id_from_url(url):
    # Extract the video ID from the URL
    return url.split('=')[1]

def get_video_details(youtube, **kwargs):
    # Get video details from YouTube API
    results = youtube.videos().list(
        part='snippet,contentDetails,statistics',
        **kwargs
    ).execute()
    return results

def get_video_snippet(youtube, video_id):
    # Get video snippet from YouTube API
    results = youtube.videos().list(
        part='snippet', id=video_id
    ).execute()
    return results

In [6]:
# Initialize an empty list to store video data
video_data = []

# Loop through each video URL in the content list
for video in content:
    # Remove any time tag from the video URL
    video_url = remove_time_tag(video)
    # Extract the video ID from the URL
    video_id = get_video_id_from_url(video_url)
    # Get video details using the YouTube API
    video_details = get_video_details(youtube, id=video_id)
    # Append the video details to the video_data list
    video_data.append(video_details)

# Write the video details to a JSON file
with open('outputs/video_details.json', 'w', encoding='utf-8') as f:
    json.dump(video_data, f, ensure_ascii=False, indent=2)
